In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
import healpy as hp

import matplotlib.pylab as pl

# Load some spherical data 

The data consists of ...

The produced maps have been down-sampled from `NSIDE=1024` to `NSIDE=64` using the `ud_grade` function of the `healpy` package.

In [ ]:
data = np.load('data/maps_downsampled_64.npz')
assert(len(data['class1'])==len(data['class2']))
nclass = len(data['class1'])

Let us plot a map of each class. It is not simple to see a difference by eye.

In [ ]:
cmin = min(np.min(data['class1']), np.min(data['class2']))
cmax = max(np.max(data['class1']), np.max(data['class2']))
hp.mollview(data['class1'][0], title='class 1, Omega_matter=0.7', nest=True,  cmap='jet', min=cmin ,max=cmax)
hp.mollview(data['class2'][0], title='class 2, Omega_matter=0.5', nest=True,  cmap='jet', min=cmin ,max=cmax)

However, those maps have different power spectral densities.

In [ ]:
def psd(x):
    '''Spherical Power Spectral Densities'''
    hatx = hp.map2alm(hp.reorder(x, n2r=True))
    p = hp.alm2cl(hatx)
    return p

In [ ]:
plt.figure()
plt.plot(psd(data['class1'][0]), label='class 1, Omega_matter=0.3, sample 0')
plt.plot(psd(data['class1'][1]), label='class 2, Omega_matter=0.5, sample 0')
plt.xscale('log')
plt.legend()
_ = pl.plot()

When averaging over all the samples, and hence getting closer to the true PSD estimate, the statistical difference becomes obvious.

In [ ]:
sample_psd_class1 = []
sample_psd_class2 = []
for i in range(nclass):
    sample_psd_class1.append(psd(data['class1'][i]))
    sample_psd_class2.append(psd(data['class2'][i]))
sample_psd_class1 = np.vstack(sample_psd_class1)
sample_psd_class2 = np.vstack(sample_psd_class2)

psd_class1 = np.mean(sample_psd_class1, axis=0)
psd_class2 = np.mean(sample_psd_class2, axis=0)

In [ ]:
plt.figure()
plt.plot(psd_class1, label='class 1, Omega_matter=0.3, mean')
plt.plot(psd_class2, label='class 2, Omega_matter=0.5, mean')
plt.xscale('log')
plt.legend()
_ = pl.plot()

# Preparing data for the classifier
Let us split the data into training and testing sets. The raw data is stored into `x_raw` and the power spectrum densities into `x_psd`.

In [ ]:
from sklearn import preprocessing


# Put all the data in a single matrix
x_raw = np.vstack((data['class1'],data['class2']))
x_raw = x_raw / np.mean(x_raw*x_raw) # Apply some normalization (We do not want to affect the mean)
x_psd = preprocessing.scale(np.vstack((sample_psd_class1,sample_psd_class2)))

# Create the label vector
labels = np.zeros([x_raw.shape[0]])
labels[len(data['class2']):] = 1
labels = labels.astype(np.int)

# Random reordering
p = np.random.permutation(len(labels))
x_raw = x_raw[p,]
x_psd = x_psd[p,]
labels = labels[p]

ntrain = 150
ntest = len(x_raw) - ntrain
x_raw_train = x_raw[:ntrain,]
x_psd_train = x_psd[:ntrain,]
labels_train = labels[:ntrain]
x_raw_test = x_raw[ntrain:,]
x_psd_test = x_psd[ntrain:,]
labels_test = labels[ntrain:]

print('Class 1 VS class 2 \n  Training set: {} / {}\n  Testing set: {} / {}'.format(
    ntrain-sum(labels_train), sum(labels_train), ntest-sum(labels_test), sum(labels_test) ))

# Classification using SVM
Let us test classify our data using an SVM classifier

An SVM based on the raw feature cannot discriminate the data because the dimensionality of the data is too large. However we observe the PSD features are linearly separable.

In [ ]:
from sklearn.svm import SVC

clf_raw = SVC(kernel='rbf')
clf_raw.fit(x_raw_train, labels_train) 

print('Errors on training: {}%'.format(
    sum(np.abs(clf_raw.predict(x_raw_train)-labels_train))/ntrain*100))
print('Errors on testing: {}%'.format(
    sum(np.abs(clf_raw.predict(x_raw_test)-labels_test))/ntest*100))

In [ ]:
clf_psd = SVC(kernel='linear')
clf_psd.fit(x_psd_train, labels_train) 

print('Errors on training: {}%'.format(
    sum(np.abs(clf_psd.predict(x_psd_train)-labels_train))/ntrain*100))
print('Errors on testing: {}%'.format(
    sum(np.abs(clf_psd.predict(x_psd_test)-labels_test))/ntest*100))

# Classification using a spherical CNN
Let us now classify our data using a spherical convolutional neural network.

In [ ]:
from scnn import models

In [ ]:
C = 2 # number of class

params = dict()
params['dir_name']       = 'test'
params['num_epochs']     = 5
params['batch_size']     = 10
params['eval_frequency'] = 10

# Building blocks.
params['brelu']          = 'b1relu' # Relu
params['pool']           = 'apool1' # Average pooling

# Architecture.
params['nsides']         = [64, 32, 16] # Sizes of the laplacians are nsides * nisides * 12
params['F']              = [5, 10, 10]  # Number of graph convolutional filters.
params['K']              = [10, 10, 10]  # Polynomial orders.
params['batch_norm']     = [True, True, True]  # Batch norm
params['M']              = [100, C]  # Output dimensionality of fully connected layers.

# Optimization.
params['regularization'] = 1e-4
params['dropout']        = 0.8
params['learning_rate']  = 1e-3
params['decay_rate']     = 0.95
params['momentum']       = 0.9
params['adam']           = True
params['decay_steps']    = ntrain / params['batch_size']

In [ ]:
model = models.scnn(**params)

In [ ]:
accuracy, loss, t_step = model.fit(x_raw_train, labels_train, x_raw_test, labels_test)

In [ ]:
print('Errors on training: {}%'.format(
    sum(np.abs(model.predict(x_raw_train)-labels_train))/ntrain*100))
print('Errors on testing: {}%'.format(
    sum(np.abs(model.predict(x_raw_test)-labels_test))/ntest*100))